In [7]:
import json,re,os,datetime,time,string
from pathlib import Path
import requests
from langchain.llms.base import LLM
from typing import *
from deepeval.models.base_model import DeepEvalBaseLLM
import traceback,random
import pandas as pd
import numpy as np
import evalplus.sanitize,evalplus.syncheck
from collections import defaultdict
import shutil
from copy import deepcopy

In [ ]:
# 连接无问芯穷的API列表
INFINI_API_List = ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]

In [ ]:
modelList = [
    'llama-3-70b-instruct',
    'llama-3-8b-instruct',
    'chatglm3',
    'chatglm2-6b',
    'chatglm2-6b-32k',
    'infini-megrez-7b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'llama-2-70b-chat',
    'llama-2-70b',
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    'chatglm3-6b',
    'chatglm3-6b-32k',
    'chatglm3-6b-base',
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b-chat',
    'qwen1.5-72b',
]
modelProfileDict = {
    'llama-3-70b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-70b-Instruct是此系列里，700亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-70b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年12月。",
    'llama-3-8b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-8b-Instruct是此系列里，80亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-8b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年3月。",
    'chatglm3':"ChatGLM3是智谱AI与清华KEG实验室发布的闭源模型，支持 8K 上下文，经过海量中英标识符的预训练与人类偏好对齐训练，相比一代模型在 MMLU、C-Eval、GSM8K 分别取得了16%、36%、280%的提升，并登顶中文任务榜单C-Eval。适用于对知识量、推理能力、创造力要求较高的场景，比如广告文案、小说写作、知识类写作、代码生成等。",
    'chatglm2-6b':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。在保留了初代模型对话流畅、部署门槛较低等众多优秀特性的基础之上，在 MMLU、C-Eval、GSM8K、BBH等主流学术数据集上，都得到了显著的性能提升，并通过基于 FlashAttention 技术，将对话模型的上下文长度（Context Length）提升至 8k tokens，允许更多轮次的对话。",
    'chatglm2-6b-32k':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。相较于ChatGLM2-6B，ChatGLM2-6b-32k支持更长的模型上下文至32k tokens。",
    'infini-megrez-7b':"由无问芯穹公司自主研发的70亿参数大语言模型。在逻辑推理、对话能力等方面有优秀的性能表现。配合无问芯穹自研高效推理引擎，同时支持Nvidia和AMD的GPU，具备更快的推理速度，在性能表现方面更上一层楼。",
    'llama-2-7b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-13b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-70b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-chat是其中700亿参数的大模型，适用于chat场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'llama-2-70b':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-base是其中700亿参数的基础大模型，适用于通用语言任务场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'baichuan2-7b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-7b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持4k tokens上下文。",
    'baichuan2-13b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持8k tokens上下文。",
    'baichuan2-13b-base':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-base是130亿参数规模的基础模型，适用于通用对话和文本续写，较chat模型更适合于复杂场景的微调后使用。该基模型支持4k tokens上下文。",
    'chatglm3-6b':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。ChatGLM3采用了全新设计的 Prompt 格式，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。模型支持 8k tokens上下文。",
    'chatglm3-6b-32k':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。相较于ChatGLM之前系列的模型，ChatGLM3采用了更多样的训练数据，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。ChatGLM3-6b-32k在ChatGLM3-6b 基础上进一步强化了对于长文本的理解能力，能够更好的处理最多32k tokens长度的上下文。",
    'chatglm3-6b-base':"ChatGLM3-6b-base 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源的基础模型。ChatGLM3-6B-Base 采用了更多样的训练数据、更充分的训练步数和更合理的训练策略。基础模型更适合于复杂场景的微调后使用，该基模型支持32k tokens上下文。",
    'qwen-7b-chat':"通义千问-7B-chat（Qwen-7B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的70亿参数规模的大语言模型。相较于Qwen-7B-Base模型，Qwen-7B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-14b-chat':"通义千问-14B-chat（Qwen-14B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的140亿参数规模的大语言模型。相较于Qwen-14B-Base模型，Qwen-14B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-72b-chat':"通义千问-72B-chat（Qwen-72B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的720亿参数规模的大语言模型。相较于Qwen-72B-Base模型，Qwen-72B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 32k tokens上下文。",
    'qwen-72b':"通义千问-72B（Qwen-72B）是阿里云研发的通义千问大模型系列的720亿参数规模的模型。Qwen-72B是基于Transformer的大语言模型, 在超大规模的预训练数据上进行训练得到。预训练数据类型多样，覆盖广泛，包括大量网络文本、专业书籍、代码等。模型支持 32k tokens上下文。",
    'qwen1.5-7b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-7b-chat是其中专用于chat场景的70亿参数的主流大小模型。",
    'qwen1.5-14b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-14b-chat是其中专用于chat场景的140亿参数的主流大小模型。",
    'qwen1.5-72b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-chat是其中专用于chat场景的720亿参数的大模型。",
    'qwen1.5-72b':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-base是其中的720亿参数的基础大模型，适合多种场景的使用。",
}
evaluateModelList = ['llama-3-70b-instruct','qwen1.5-72b-chat',]
# 测评模型列表
answerModelList = [    
    'llama-3-8b-instruct',
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    'infini-megrez-7b', 
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b',
    'llama-2-70b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'llama-2-70b-chat'
]

# KoLA

In [ ]:
knowledgeList = ['Knowledge Memorization','Knowledge Understanding','Knowledge Applying','Knowledge Creating']

### DataProcess

##### Get Model Reply

In [ ]:
index = 0
def LLMCompletions(prompt:str,modelName:str = "infini-megrez-7b",INFINI_API_List:List[str] = ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"],returnContent:bool = True,**kwargs)->str:
    """_summary_
        invoke the model `modelName` with the `prompt` and configuration in kwargs to get the reply
    Args:
        prompt (str): question profile
        modelName (str, optional): the model name that will be called. Defaults to "infini-megrez-7b".
        INFINI_API_List (List[str], optional): _description_. Defaults to ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"].
        returnContent (bool, optional): whether return the model reply string directly or not. Defaults to True.

    Returns:
        str: return the model reply string 
    """
    global index
    url = "https://cloud.infini-ai.com/maas/"+modelName+"/nvidia/chat/completions"
    payload = {
        "model": modelName,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "temperature": kwargs["temperature"] if "temperature" in kwargs else 0.7,
        "top_p": kwargs["top_p"] if 'top_p' in kwargs else 1,
        "top_k": kwargs['top_k'] if 'top_k' in kwargs else -1,
        "n": kwargs['n'] if 'n' in kwargs else 1,
        "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else None,
        "stop": kwargs['stop'] if 'stop' in kwargs else None,
        "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else 0,
        "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else 0
    }
    idx = 0
    while idx < len(INFINI_API_List):
        headers = {
                'Content-Type': "application/json",
                'Accept': "*/*",
                'Authorization': "Bearer "+INFINI_API_List[index%len(INFINI_API_List)],
        } 
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            response.encoding = 'utf-8'
            data = response.json()
            content = data['choices'][0]['message']['content']
            if isinstance(content,str):
                content = content.replace(',\n}','\n}')
                content = content.replace(']\n}',']}')
                content = content.replace('\\','\\\\')
            if returnContent:
                return content
            try:
                content = json.loads(content)
            except:
                pass
            data['choices'][0]['message']['content'] = content
            if isinstance(content,str):
                return content
            
            return json.dumps(data['choices'][0]['message']['content'])
        else:
            print(response.status_code)
            try:
                print(response.json())
            except:
                pass
        index = (index + 1) % len(INFINI_API_List)
        idx += 1
    return "Cannot connect to the model "+modelName

###### the first time to get the model reply 

In [ ]:
data_format_ins = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'time':-1
}
def get_LLM_Reply_KoLA(filepath:Union[str,Path],savePath:Union[str,Path],errorPath:Union[str,Path],fileName:str = None,)->None:
    """_summary_
        get evaluation questions from `filepath` and save the reply of the model in answerModelList to `savePath` ,and error item to `errorPath`
    Args:
        filepath (Union[str,Path]): _description_
        savePath (Union[str,Path]): the path to save LLM reply
        errorPath (Union[str,Path]): the path to save error items
        fileName (str, optional): _description_. Defaults to None.
    """
    with open(filepath) as f:
        data = json.load(f)
    instructions = data['adapter_spec']['instructions']
    questionList = data['request_states']
    errorItem = []
    if not fileName:
        fileName = Path(filepath).name
    save = {'fileName':fileName,'class':knowledgeList[int(fileName[0])-1],'data':[]}
    for index,item in enumerate(questionList):
        if 'id' in item['instance']:
            data_format_ins['id'] = item['instance']['id']
        else:
            data_format_ins['id'] = fileName+'%04d'% index
        prompt = instructions+'\n'+item['instance']['input']['text']
        data_format_ins['input'] = prompt
        print(prompt)
        if item['instance']['references'][0]['tags'][0] == 'correct':
            data_format_ins['expected_output'] = item['instance']['references'][0]['output']['text']
        else:
            data_format_ins['expected_output'] = None
        for model in answerModelList:
            data_format_ins['AnswerModel'] = model
            print(model)
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model)
            end = time.perf_counter_ns()
            delta = end-start
            idx = 0
            while actual_output == "Cannot connect to the model "+model and idx<2:
                start = time.perf_counter_ns()
                actual_output =  LLMCompletions(prompt,modelName=model)
                end = time.perf_counter_ns()
                delta = end-start
                idx += 1
            if actual_output == "Cannot connect to the model "+model:
                
                errorItem.append({'fileName':fileName,'id':data_format_ins['id'],"AnswerModel":model,"input":prompt,"expected_output":data_format_ins['expected_output']})
                continue
            print(idx,delta,actual_output,sep='\t')
            data_format_ins['actual_output'] = actual_output
            data_format_ins['time'] = delta
            save['data'].append(data_format_ins.copy())
            print('*'*70)
        print('+'*70)
    errorItemFinal = []
    while errorItem:
        item = errorItem.pop()
        data_format_ins['id'] = item['id']
        model = item['AnswerModel']
        data_format_ins['AnswerModel'] = model
        prompt = item['input']
        data_format_ins['input'] = prompt
        data_format_ins['expected_output'] = item['expected_output']
        start = time.perf_counter_ns()
        actual_output =  LLMCompletions(prompt,modelName=model)
        end = time.perf_counter_ns()
        delta = end-start
        idx = 0
        while actual_output == "Cannot connect to the model "+model and idx<2:
            print('\t'+str(idx)+'\ttest')
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model)
            end = time.perf_counter_ns()
            delta = end-start
            idx += 1
        if actual_output == "Cannot connect to the model "+model:
            errorItemFinal.append(item)
            print("[error]:\t"+str(errorItemFinal[-1]))
            continue
        print(idx,delta,actual_output,sep='\t')
        data_format_ins['actual_output'] = actual_output
        data_format_ins['time'] = delta
        save['data'].append(data_format_ins.copy())
    with open(savePath,'w',encoding='utf-8') as out:
        json.dump(save,out)
    if errorItemFinal:
        error = {'fileName':fileName,'class':knowledgeList[int(fileName[0])-1],'data':[]}
        for i in errorItemFinal:
            error['data'].append(i)
        with open(errorPath,'w',encoding='utf-8') as out:
            json.dump(error,out)
    print('='*70)

In [2]:
for dirName,subDirName,fileNames in os.walk('E:\\Repository\\KoLA\\Sample_Data'):
    print(fileNames)

['1-1_2_high_freq_ent_sample.json', '1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json', 'README.md']


In [ ]:
fileNames =['1-1_2_high_freq_ent_sample.json', '1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json']
dirName = 'E:/Repository/KoLA/Sample_Data'

In [ ]:
for fileName in fileNames:
    get_LLM_Reply_KoLA(os.path.join(dirName, fileName),'./data/KoLA/save/'+fileName,'./data/KoLA/error/'+fileName.replace(".json","")+'Error'+'.json',fileName)

###### handle error items

In [ ]:
data_format_ins = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'time':-1
}
def joinErrorToData(errorFile:Union[str,Path],saveFile:Union[str,Path])->None:
    """rerun the error items in `saveFile` and append results to 'saveFile'

    Args:
        errorFile (Union[str,Path]): 
        saveFile (Union[str,Path]): 
    """
    with open(errorFile, 'r') as ef:
        data_ef = json.load(ef)
    fileName = data_ef['fileName']
    with open(saveFile) as sf:
        data_sf = json.load(sf)
    if fileName != data_sf['fileName']:
        print('FileName not match')
        return 
    if not data_ef['data']:
        print('The Errors of this ErrorFile all have been solved!')
        return 
    errorItem = []
    for item in data_ef['data']:
        data_format_ins['id'] = item['id']
        model = item['AnswerModel']
        data_format_ins['AnswerModel'] = model
        prompt = item['input']
        data_format_ins['input'] = prompt
        data_format_ins['expected_output'] = item['expected_output']
        start = time.perf_counter_ns()
        actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
        end = time.perf_counter_ns()
        delta = end-start
        idx = 0
        while actual_output == "Cannot connect to the model "+model and idx<2:
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
            end = time.perf_counter_ns()
            delta = end-start
            idx += 1
        if actual_output == "Cannot connect to the model "+model:
            errorItem.append(item)
            print("[error]:\t"+str(errorItem[-1]))
            continue
        print(actual_output)
        data_format_ins['time'] = delta
        data_format_ins['actual_output'] = actual_output
        data_sf['data'].append(data_format_ins.copy())
    data_ef['data'] = errorItem
    with open(saveFile,'w') as saveF:
        json.dump(data_sf,saveF)
    with open(errorFile,'w') as error:
        json.dump(data_ef,error)
    if errorItem:
        print("There are still some errors! ")
    else:
        print('The Errors of this ErrorFile all have been solved!')

In [3]:
# joinErrorToData('./data/KoLA/error/4-2_r_with_triples_sampleError.json','./data/KoLA/save/4-2_r_with_triples_sample.json')
for dir,subdir,files in os.walk("./data/KoLA/error"):
    for file in files:
        joinErrorToData(os.path.join(dir,file),os.path.join('./data/KoLA/save',file.replace("Error.json",".json")))

In [ ]:
def removeDeprecatedModel(filePath:Union[str,Path]):
    """remove the deprecated evaluated model  from the `filePath`

    Args:
        filePath (Union[str,Path]): 
    """
    with open(filePath) as f:
        data = json.load(f)
    new_data = []
    for item in data['data']:
        if item['AnswerModel']  in answerModelList:
            new_data.append(item)
    data['data'] = new_data[:]
    with open(filePath,'w') as f:
        json.dump(data,f)

In [ ]:
# removeDeprecatedModel('./data/1-2_1_low_freq_ent_sample.json')

### Evaluation

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric,FaithfulnessMetric,HallucinationMetric,BaseMetric
from deepeval.test_case import LLMTestCase

In [ ]:
class ChatLLM(LLM):
    @property
    def modelName(self)->str:
        return "qwen1.5-72b-chat"
    @property
    def INFINI_API_List(self)->List[str]:
        return ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]
    @property
    def temperature(self)->float:
        return 0.7
    @property
    def top_p(self)->float:
        return 0.1
    @property
    def top_k(self)->int:
        return -1
    @property
    def n(self)->int:
        return 1
    @property
    def max_tokens(self)->int:
        return None
    @property
    def stop(self)->Optional[List[str]]:
        return None
    @property
    def presence_penalty(self)->float:
        return 0
    @property
    def frequency_penalty(self)->float:
        return 0
    def getHeader(self,index_api):  
        headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+self.INFINI_API_List[index_api%len(self.INFINI_API_List)],
        }
        return headers
    @property
    def _llm_type(self)->str:
        return "ChatLLM"
    @property
    def _identifying_params(self)->Mapping[str,Any]:
        _param_dict = {
            "modelName":self.modelName,
            "INFINI_API":self.getHeader(self.__fields__['index_api'] if 'index_api' in self.__fields__ else 0),
            "stream":bool(self.stream),
            "temperature":self.temperature,
            "top_p":self.top_p,
            "top_k":self.top_k,
            "n":self.n,
            "max_tokens":self.max_tokens,
            "stop":self.stop,
            "presence_penalty":self.presence_penalty,
            "frequency_penalty":self.frequency_penalty,
        }
        return _param_dict

    def _call(self, prompt: str, stop: Optional[List[str]]= None, run_manager= None,**kwargs: Any) -> str:
        url = "https://cloud.infini-ai.com/maas/"+str(self.modelName)+"/nvidia/chat/completions"
        payload = {
            "model": "string",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False,
            "temperature": kwargs["temperature"] if "temperature" in kwargs else self.temperature,
            "top_p": kwargs["top_p"] if 'top_p' in kwargs else self.top_p,
            "top_k": kwargs['top_k'] if 'top_k' in kwargs else self.top_k,
            "n": kwargs['n'] if 'n' in kwargs else self.n,
            "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens,
            "stop": stop if stop else self.stop,
            "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty,
            "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
        }
        index = 0
        if 'index_api' not in self.__fields__:
            self.__fields__['index_api'] = -1
        index_api = self.__fields__['index_api']+1
        length = len(self.INFINI_API_List)
        while index < length:
            response = requests.post(url, json=payload, headers=self.getHeader(index_api))
            if response.status_code == 200:
                response.encoding = 'utf-8'
                data = response.json()
                print("response json success")
                content = data['choices'][0]['message']['content']
                if isinstance(content,str):   
                    content = content.replace(',\n}','\n}')
                    content = content.replace(']\n}',']}')
                    if 'statements' in content:
                        regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
                        matchStr =regex.search(content)
                        if matchStr:
                            content = '{'+matchStr.group()
                    elif 'verdicts' in content:
                        regex = re.compile('\"verdicts\":\s+\[.*\]\}',re.DOTALL)
                        matchStr =regex.search(content)
                        if matchStr is not None:
                            content ='{' +matchStr.group()
                            regex = re.compile("\"reason\":(.*?)\}",re.DOTALL)
                            matchStr = regex.findall(content)
                            for string in matchStr:
                                tmp = string.strip()[1:-1].replace('"','\\\"')
                                tmp = '\"'+tmp+"\""
                                content = content.replace(string,tmp)
                if isinstance(content,str):
                    return content
                data['choices'][0]['message']['content'] = content
                return json.dumps(data['choices'][0]['message']['content'])

            index += 1
            index_api =  (index_api+1)%length
            self.__fields__['index_api'] = index_api
            print(response.status_code)
            try:
                print(response.json())
            except:
                pass
            time.sleep(1)
        return "Cannot connect to the model "+self.modelName
    def setParameter(self,**kwargs):
        self.temperature = kwargs["temperature"] if "temperature" in kwargs else self.temperature
        self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else self.top_p
        self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else self.top_k
        self.n = kwargs['n'] if 'n' in kwargs else self.n
        self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens
        self.stop = kwargs['stop'] if 'stop' in kwargs else self.stop
        self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty
        self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
    

class CustomLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model
    def load_model(self):
        return self.model
    def generate(self, prompt: str) -> str:
        # global path
        chat_model = self.load_model()
        ret = chat_model.invoke(prompt)
        idx = 0
        while ret == "Cannot connect to the model "+self.get_model_name() and idx<5:
            time.sleep(5)
            ret = chat_model.invoke(prompt)
            idx += 1
        # print(ret)
        # path.append(ret)
        return ret
    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        try:
            return self.model.modelName
        except:
            return "CustomLLM"
custom_model = ChatLLM()
evaluateModel = CustomLLM(model=custom_model)

In [ ]:
metrics_format_example = {
    'metric_metadata':{
        'metric':None,
        'threshold':0,
        'success':True,
        'score':0.8,
        'reason':'',
        'strictMode': False,
        'evaluationModel': 'CustomLLM',
        'evaluationCost': 0
    },
    'metric_configuration': {
        'threshold': 0.5,
        'evaluation_model': 'CustomLLM',
        'strict_mode': False,
        'include_reason': True
    }
}

data_format_example = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'cached_metrics_data':[
        {
            'metric_metadata':{
                'metric':None,
                'success':True,
                'score':0.8,
                'reason':'',
                'statements':'',
                'verdicts':'',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        },
        metrics_format_example
    ]
}


In [ ]:
def evaluate_file(filename:Union[str,Path],save_file:Union[str,Path],error_file:Union[str,Path],force_save:bool = False,metrics:List[BaseMetric] = [AnswerRelevancyMetric(threshold=0.5,model=evaluateModel,include_reason=True),HallucinationMetric(threshold=0.5,model=evaluateModel,include_reason=True)]):
    """_summary_
        the function is used to evaluate the LLM output saved in `filename` by the metric in `metrics`,the successful eval results will be saved into `save_file` and the error item will be saved into `error_file`
    
    Args:
        `filename` (Union[str,Path]): the filename saves the LLM generation results
        `save_file` (Union[str,Path]): the filename will save the evaluate results
        `error_file` (Union[str,Path]): the filename will save the error eval item
        `force_save` (bool, optional): if the value is `True`,function will rerun all eval item in `filename` and directly override the `save_file` and `error_file`. 
                    Defaults to False.
        `metrics` (List[BaseMetric], optional): a list of evaluation metrics. 
                    Defaults to [AnswerRelevancyMetric(threshold=0.5,model=evaluateModel,include_reason=True),HallucinationMetric(threshold=0.5,model=evaluateModel,include_reason=True)].
    """
    def is_same_eval_item(x,item):
        if x['id'] == item['id']  and x['AnswerModel'] == item['AnswerModel'] :
            return True
        return False
    with open(filename,'r') as f:
        data = json.load(f)
    save,error = dict(),dict()
    if  Path(save_file).is_file() and not force_save:
        with open(save_file) as f:
            save = json.load(f)
        if 'fileName' in save and  save['fileName'] != data['fileName']:
            print("The save_file does not match the file name!")
            return
    else:
        save = {'fileName':data['fileName'],'class':data['class'],'data':[]}

    if Path(error_file).is_file() and not force_save:
        with open(error_file) as f:
            error = json.load(f)
        if 'fileName' in error and  error['fileName'] != data['fileName']:
            print("The error_file does not match the file name!")
            return
    else:
        error = {'fileName' :data['fileName'],'class':data['class'],'data':[]}
    for metric in metrics:
        for item in data['data']:
            data_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'retrieval_context':None,
                'cached_metrics_data':[]
            }
            metrics_format_ins = {
                'metric_metadata':{
                    'metric':None,
                    'success':True,
                    'score':0.8,
                    'reason':'',
                    'statements':'',
                    'verdicts':'',
                    'evaluationCost': 0
                },
                'metric_configuration': {
                    'threshold': 0.5,
                    'evaluation_model': 'CustomLLM',
                    'strict_mode': False,
                    'include_reason': True
                }
            }
            errors_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'retrieval_context':None,
                'cached_metrics_data':[
                    {            
                        'metric_metadata':{
                            'metric':None,
                        },
                        'metric_configuration': {
                            'threshold': 0.5,
                            'evaluation_model': 'CustomLLM',
                            'strict_mode': False,
                            'include_reason': True
                        }
                    }
                ]
            }
            
            data_format_ins['id'] = item['id']
            data_format_ins['AnswerModel'] = item['AnswerModel']
            data_format_ins['input'] = item['input']
            data_format_ins['actual_output'] = item['actual_output']
            data_format_ins['expected_output'] = item['expected_output']
            tag = False
            for x in save['data']:
                if is_same_eval_item(item,x):
                    for metric_data in x['cached_metrics_data']:
                        if metric_data['metric_metadata']['metric'] == metric.__name__:
                            if metric_data['metric_configuration']['threshold'] == metric.threshold and metric_data['metric_configuration']['evaluation_model'] == metric.evaluation_model and metric_data['metric_configuration']['strict_mode'] == metric.strict_mode and metric_data['metric_configuration']['include_reason'] == metric.include_reason:
                                tag = True
                                print("HAVE:")
                                print(x)
                                break
            if tag:
                continue
            test_case = LLMTestCase(
                input= item['input'],
                actual_output=item['actual_output'],
                context=[item['expected_output']],
            )
            try:
                metric.measure(test_case)
            except Exception as e:
                print(e)
                errors_format_ins['id'] = item['id']
                errors_format_ins['AnswerModel'] = item['AnswerModel']
                errors_format_ins['input'] = item['input']
                errors_format_ins['actual_output'] = item['actual_output']
                errors_format_ins['expected_output'] = item['expected_output']
                errors_format_ins['cached_metrics_data'][0]['metric_metadata']['metric'] = metric.__name__
                errors_format_ins['cached_metrics_data'][0]['metric_configuration'] = {'threshold':metric.threshold,'evaluation_model':metric.evaluation_model,'strict_mode':metric.strict_mode,'include_reason':metric.include_reason}
                error['data'].append(errors_format_ins.copy())
                print(errors_format_ins)
                with open(error_file,'w') as f:
                    json.dump(error,f,indent=4)
                continue
            metrics_format_ins['metric_metadata']['metric'] = metric.__name__
            metrics_format_ins['metric_metadata']['score'] = metric.score
            metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
            metrics_format_ins['metric_metadata']['reason'] = metric.reason
            
            metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
            metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
            metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

            metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
            metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
            metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
            metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason

            data_format_ins['cached_metrics_data'].append(metrics_format_ins.copy())
            flag = True
            for each in save['data']:
                if is_same_eval_item(each, data_format_ins):
                    each['cached_metrics_data'].append(metrics_format_ins.copy())
                    flag = False
                    break
            if flag:
                save['data'].append(data_format_ins.copy())
                print(data_format_ins)
            with open(save_file,'w') as f:
                json.dump(save,f,indent=4)

In [ ]:
for file in os.listdir('./data/KoLA/save/'):
    evaluate_file('./data/KoLA/save/'+file,'./data/KoLA/eval/save/'+file,'./data/KoLA/eval/error/'+file)

In [ ]:
def joinEvalErrorToData(errorFile:Union[str,Path],saveFile:Union[str,Path])->None:
    """_summary_
        the function is used to rerun the error item in the `errorFile` and append the results into the `saveFile` 
        
    Args:
        `errorFile` (Union[str,Path]): the JSON file saves the error item in the before running
        `saveFile` (Union[str,Path]): the JSON file saves the pass result
        
    Returns:
        None: the result will override the original file 
    """
    def is_same_eval_item(x,item):
        if x['id'] == item['id'] and x['AnswerModel'] == item['AnswerModel'] :
            return True
        return False
    with open(errorFile,'r') as f:
        data_er = json.load(f)
    with open(saveFile,'r') as f:
        data_sv = json.load(f)
    if data_er['fileName'] != data_sv['fileName']:
        print("The save_file does not match the error_file!")
        return
    error = {'fileName' :data_er['fileName'],'class':data_er['class'],'data':[]}
    while data_er['data']:
        item = data_er['data'].pop()
        data_format_ins = {
            'id':0,
            'AnswerModel':'',
            'input':'',
            'actual_output':'',
            'expected_output':None,
            'context':None,
            'retrieval_context':None,
            'cached_metrics_data':[]
        }
        metrics_format_ins = {
            'metric_metadata':{
                'metric':None,
                'success':True,
                'score':0.8,
                'reason':'',
                'statements':'',
                'verdicts':'',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        }
        errors_format_ins = {
            'id':0,
            'AnswerModel':'',
            'input':'',
            'actual_output':'',
            'expected_output':None,
            'retrieval_context':None,
            'cached_metrics_data':[
                {            
                    'metric_metadata':{
                        'metric':None,
                    },
                    'metric_configuration': {
                        'threshold': 0.5,
                        'evaluation_model': 'CustomLLM',
                        'strict_mode': False,
                        'include_reason': True
                    }
                }
            ]
        }
        data_format_ins['id'] = item['id']
        data_format_ins['AnswerModel'] = item['AnswerModel']
        data_format_ins['input'] = item['input']
        data_format_ins['actual_output'] = item['actual_output']
        data_format_ins['expected_output'] = item['expected_output']
        
        tag = False
        for x in data_sv['data']:
            if is_same_eval_item(item,x):
                for metric_data in x['cached_metrics_data']:
                    if metric_data['metric_metadata']['metric'] == item['cached_metrics_data'][0]['metric_metadata']['metric']:
                        if metric_data['metric_configuration']['threshold'] == item['cached_metrics_data'][0]['metric_configuration']['threshold'] and metric_data['metric_configuration']['evaluation_model'] == item['cached_metrics_data'][0]['metric_configuration']['evaluation_model'] and metric_data['metric_configuration']['strict_mode'] == item['cached_metrics_data'][0]['metric_configuration']['strict_mode'] and metric_data['metric_configuration']['include_reason'] == item['cached_metrics_data'][0]['metric_configuration']['include_reason']:
                            tag = True
                            print("HAVE:")
                            print(x)
                            break
        if tag:
            continue
        test_case = LLMTestCase(
            input= item['input'],
            actual_output=item['actual_output'],
            context=[item['expected_output']],
        )
        if item['cached_metrics_data'][0]['metric_metadata']['metric'] == 'Answer Relevancy':
            metric = AnswerRelevancyMetric(
                threshold=item['cached_metrics_data'][0]['metric_configuration']['threshold'],
                model = evaluateModel,
                include_reason=item['cached_metrics_data'][0]['metric_configuration']['include_reason']
            )
        elif item['cached_metrics_data'][0]['metric_metadata']['metric'] == 'Hallucination':
            metric = HallucinationMetric(
                threshold=item['cached_metrics_data'][0]['metric_configuration']['threshold'],
                model = evaluateModel,
                include_reason=item['cached_metrics_data'][0]['metric_configuration']['include_reason']
            )
        else:
            print("unkonwn metric!")
            continue
        
        try:
            metric.measure(test_case)
        except Exception as e:
            print(traceback.print_exc())
            errors_format_ins['id'] = item['id']
            errors_format_ins['AnswerModel'] = item['AnswerModel']
            errors_format_ins['input'] = item['input']
            errors_format_ins['actual_output'] = item['actual_output']
            errors_format_ins['expected_output'] = item['expected_output']
            errors_format_ins['cached_metrics_data'][0]['metric_metadata']['metric'] = metric.__name__
            errors_format_ins['cached_metrics_data'][0]['metric_configuration'] = {'threshold':metric.threshold,'evaluation_model':metric.evaluation_model,'strict_mode':metric.strict_mode,'include_reason':metric.include_reason}
            error['data'].append(errors_format_ins.copy())
            print(errors_format_ins)
            with open(errorFile,'w') as f:
                json.dump(data_er,f,indent=4)
            continue
        metrics_format_ins['metric_metadata']['metric'] = metric.__name__
        metrics_format_ins['metric_metadata']['score'] = metric.score
        metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
        metrics_format_ins['metric_metadata']['reason'] = metric.reason
        
        metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
        metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
        metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

        metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
        metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
        metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
        metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason

        data_format_ins['cached_metrics_data'].append(metrics_format_ins.copy())
        flag = True
        for each in data_sv['data']:
            if is_same_eval_item(each, data_format_ins):
                each['cached_metrics_data'].append(metrics_format_ins.copy())
                flag = False
                break
        if flag:
            data_sv['data'].append(data_format_ins.copy())
            print(data_format_ins)
        with open(saveFile,'w') as f:
            json.dump(data_sv,f,indent=4)
    
    with open(errorFile,'w') as f:
        json.dump(error,f,indent=4)

In [ ]:
for dir,subdirs,files in os.walk("./data/KoLA/eval/error"):
    for file in files:
        if file.endswith(".json"):
            joinEvalErrorToData('./data/KoLA/eval/error/'+file,'./data/KoLA/eval/save/'+file)

In [ ]:
files_error = ['1-1_2_high_freq_ent_sample.json','1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json',  '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-4_kqapro_sample.json',  '3-6_r_KoRC++ood_sample.json',  '4-1_with_triples_sample.json', '4-2_r_with_triples_sample.json']
for file in files_error[::-1]:
    joinEvalErrorToData('./data/KoLA/eval/error/'+file,'./data/KoLA/eval/save/'+file)

In [ ]:
class UnionFind:
    def __init__(self,n):
        self.n = n
        self.parent = [i for i in range(n)]
        self.size = n
        self.keyset = [1]*n
    def find(self,x):
        if self.parent[x]!= x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self,x,y):
        x = self.find(x)
        y = self.find(y)
        if x == y:
            return False
        if self.keyset[x] < self.keyset[y]:
            x,y = y,x
        self.parent[y] = x
        self.keyset[x] += self.keyset[y]
        self.size -= 1
        return True
    
    def is_connected(self,x,y):
        return self.find(x) == self.find(y)
    def get_size(self,x):
        return self.keyset[self.find(x)]
    def get_size_all(self):
        return self.size
def merge_same_item(file_path:Union[str,Path])->None:
    def is_same_eval_item(x,item):
        if x['id'] == item['id'] and  x['AnswerModel'] == item['AnswerModel'] :
            return True
        return False
    with open(file_path,'r') as f:
        data = json.load(f)
    check = []
    for item in data['data']:
        if len(item["cached_metrics_data"]) <2:
            check.append(item)
    check.sort(key=lambda x:(x['id'],x['AnswerModel']))
    uf = UnionFind(len(check))
    for i in range(len(check)):
        for j in range(i+1,len(check)):
            if is_same_eval_item(check[i],check[j]):
                uf.union(i,j)
    key_set = defaultdict(list)
    for i in range(len(check)):
        p = uf.find(i)
        key_set[p].append(check[i])
        
    mergeList =  list(key_set.values())
    for item in mergeList[:]:
        if len(item)<2:
            mergeList.remove(item)
    for x,y in mergeList:
        mergeItem = x.copy()
        mergeItem['cached_metrics_data'].append(y['cached_metrics_data'][0])
        data['data'].remove(x)
        data['data'].remove(y)
        data['data'].append(mergeItem)
    with open(file_path,'w') as f:
        json.dump(data,f,indent=4)
    print(file_path)

In [ ]:
for dir,subdir,files in os.walk('./data/KoLA/eval/save'):
    for file in files:
        merge_same_item('./data/KoLA/eval/save/'+file)

### ADD `idx` To replace `id`

###### 后来发现原始数据中的`id`存在问题，故使用`idx`进行替代

In [ ]:
for dir,subdir,files in os.walk("E:\\Repository\\KoLA\\Sample_Data"):
    for file in files:
        if file.endswith(".json"):
            with open(os.path.join(dir,file)) as f:
                data = json.load(f)
            instructions = data["adapter_spec"]["instructions"]
            prefix = file.replace(".json","").replace("_sample","")
            for i,ins in enumerate(data["request_states"]):
                ins["instance"]["idx"] = prefix+"=="+("%02d"%i)
            with open(os.path.join(dir,file),'w') as f:
                json.dump(data,f,indent=4)

In [ ]:
dir_o = "E:\\Repository\\KoLA\\Sample_Data"
dir_data = "E:\\学习资料\\2023-2024第二学期_大二下\\信息存储与检索\\小组作业\\data\\KoLA\\save"
dir_eval = "E:\\学习资料\\2023-2024第二学期_大二下\\信息存储与检索\\小组作业\\data/KoLA/eval\\save"
dir_eval_error = "E:\\学习资料\\2023-2024第二学期_大二下\\信息存储与检索\\小组作业\\data/KoLA/eval\\error"
fileList = ['1-1_2_high_freq_ent_sample.json', '1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json']
for file in fileList:
    with open(os.path.join(dir_o,file)) as f:
        data_origin = json.load(f)
    # dir_data = dir_eval
    # dir_data = dir_eval_error
    if not os.path.exists(os.path.join(dir_data,file)):
        continue
    with open(os.path.join(dir_data,file)) as f:
        data_save = json.load(f)
    if data_save['fileName'] != file:
        print("Not Match "+file)
        continue
    instructions = data_origin["adapter_spec"]["instructions"]
    for ins in data_origin["request_states"]:
        inp = ins["instance"]["input"]["text"]
        idx = ins["instance"]["idx"]
        expected_output = ins["instance"]["references"][0]["output"]["text"]
        for item in data_save['data']:
            if item["expected_output"] == expected_output and item["input"] == instructions+"\n"+inp :
                item["idx"] = idx
    with open(os.path.join(dir_data,file),'w') as f:
        json.dump(data_save,f,indent=4)

### Evaluation Results View

In [3]:
data_eval = {"Answer Relevancy":defaultdict(list),"Hallucination":defaultdict(list),}
for dir,subdir,files in os.walk('data/KoLA/eval/save'):
    for file in files:
        with open(os.path.join(dir,file)) as f:
            data = json.load(f)
        for item in data['data']:
            for metric in item["cached_metrics_data"]:
                data_eval[metric["metric_metadata"]["metric"]][item['AnswerModel']].append(metric["metric_metadata"]["score"])


In [4]:
data_eval

{'Answer Relevancy': defaultdict(list,
             {'baichuan2-13b-base': [0.5,
               0.5,
               1.0,
               0.5,
               0.5,
               0.5,
               0.0,
               0.0,
               0.5,
               0.0,
               1.0,
               0.8,
               0.5,
               0.4,
               1.0,
               1.0,
               1.0,
               0.5714285714285714,
               0.21428571428571427,
               1.0,
               0.0,
               1.0,
               1.0,
               1.0,
               1.0,
               1.0,
               0.6666666666666666,
               1.0,
               0.0,
               0.0,
               1.0,
               0.0,
               1.0,
               1.0,
               0.6666666666666666,
               0.3333333333333333,
               0.0,
               1.0,
               0.25,
               0.7777777777777778,
               1.0,
               1.0,
       

In [5]:
data_eval_mean = {"Answer Relevancy":defaultdict(dict),"Hallucination":defaultdict(dict),}
for metric,modelEvalItem in data_eval.items():
    for model,evals in modelEvalItem.items():
        data_eval_mean[metric][model]["mean"] = np.mean(evals)
        data_eval_mean[metric][model]["variance"] = np.var(evals)

In [6]:
data_eval_mean

{'Answer Relevancy': defaultdict(dict,
             {'baichuan2-13b-base': {'mean': 0.6114362741716677,
               'variance': 0.16698578079239057},
              'baichuan2-13b-chat': {'mean': 0.9458282458282459,
               'variance': 0.015045496292850789},
              'baichuan2-7b-chat': {'mean': 0.8908861703514317,
               'variance': 0.06627432625785984},
              'infini-megrez-7b': {'mean': 0.8334527612305389,
               'variance': 0.09087302114851438},
              'llama-2-13b-chat': {'mean': 0.886645419945961,
               'variance': 0.05291497173885806},
              'llama-2-70b': {'mean': 0.6454967130863714,
               'variance': 0.16926455695663895},
              'llama-2-70b-chat': {'mean': 0.9011196789321788,
               'variance': 0.04659675264952888},
              'llama-2-7b-chat': {'mean': 0.8812358470896451,
               'variance': 0.06371954794626675},
              'llama-3-8b-instruct': {'mean': 0.9317500738553369,


In [ ]:

data_view = {}
for dir,subdir,files in os.walk('data/KoLA/eval/save'):
    for file in files:
        data_id = defaultdict(dict)
        with open(os.path.join(dir,file)) as f:
            data = json.load(f)
        for item in data['data']:
            for metric in item["cached_metrics_data"]:
                if metric["metric_metadata"]["metric"] not in data_id[item['AnswerModel']]:
                    data_id[item['AnswerModel']][metric["metric_metadata"]["metric"]] = []
                data_id[item['AnswerModel']][metric["metric_metadata"]["metric"]].append(metric["metric_metadata"]["score"])
        for key,value in data_id.items():
            for k,v in value.items():
                data_id[key][k] = np.mean(v)
        data_view[file] = data_id.copy()

data_view_ar = deepcopy(data_view)
data_view_h = deepcopy(data_view)
for key,value in data_view_ar.items():
    for k,v in value.items():
        data_view_ar[key][k] = v['Answer Relevancy'] if 'Answer Relevancy' in v else None
        data_view_h[key][k] = v['Hallucination'] if 'Hallucination' in v else None
        
# pd.DataFrame(data_view).to_json("data/KoLA/results_view/model_file_dict_AR_H.json",indent=4)
# pd.DataFrame(data_view_ar).to_json("data/KoLA/results_view/model_file_AnswerRelevancy.json",indent=4)
# pd.DataFrame(data_view_h).to_json("data/KoLA/results_view/model_file_Hallucination.json",indent=4)
# pd.DataFrame(data_view).to_excel("data/KoLA/results_view/model_file_dict_AR_H.xlsx")
# pd.DataFrame(data_view_ar).to_excel("data/KoLA/results_view/model_file_AnswerRelevancy.xlsx",)
# pd.DataFrame(data_view_h).to_excel("data/KoLA/results_view/model_file_Hallucination.xlsx",)

In [ ]:
pd.DataFrame(data_view)

,1-1_2_high_freq_ent_sample.json,1-2_1_low_freq_ent_sample.json,1-3_r_1_simple_sample_sample.json,2-1_COPEN++csj_sample.json,2-2_COPEN++cpj_sample.json,2-3_COPEN++cic_sample.json,2-4_FewNERD++inter_sample.json,2-4_FewNERD++intra_sample.json,2-4_FewNERD++supervised_sample.json,2-5_DocRED_sample.json,...,3-1_hotpotqa_sample.json,3-2_2wikimultihopqa_sample.json,3-3_musique_sample.json,3-4_kqapro_sample.json,3-5_KoRC++ood_sample.json,3-6_r_KoRC++ood_sample.json,4-1_without_triples_sample.json,4-1_with_triples_sample.json,4-2_r_without_triples_sample.json,4-2_r_with_triples_sample.json
baichuan2-13b-base,"{'Answer Relevancy': 0.6, 'Hallucination': 0.6}","{'Answer Relevancy': 0.25, 'Hallucination': 0.6}","{'Answer Relevancy': 0.5399999999999999, 'Hall...","{'Answer Relevancy': 0.7571428571428571, 'Hall...","{'Answer Relevancy': 0.8, 'Hallucination': 0.6}","{'Answer Relevancy': 0.7333333333333333, 'Hall...","{'Answer Relevancy': 0.6, 'Hallucination': 0.8}","{'Answer Relevancy': 0.45, 'Hallucination': 0.8}","{'Answer Relevancy': 0.6888888888888889, 'Hall...","{'Answer Relevancy': 0.625, 'Hallucination': 1.0}",...,"{'Answer Relevancy': 0.4066666666666666, 'Hall...","{'Answer Relevancy': 0.4552631578947368, 'Hall...","{'Answer Relevancy': 0.7, 'Hallucination': 1.0}","{'Answer Relevancy': 0.3333333333333333, 'Hall...","{'Answer Relevancy': 0.6285714285714286, 'Hall...","{'Answer Relevancy': 0.375, 'Hallucination': 0.6}","{'Answer Relevancy': 1.0, 'Hallucination': 0.2}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.4833333333333333, 'Hall...","{'Answer Relevancy': 0.8, 'Hallucination': 0.6}"
baichuan2-13b-chat,"{'Answer Relevancy': 0.95, 'Hallucination': 0....","{'Answer Relevancy': 1.0, 'Hallucination': 0.6}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.96, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.2}","{'Answer Relevancy': 1.0, 'Hallucination': 0.6}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.9777777777777779, 'Hall...","{'Answer Relevancy': 0.8833333333333332, 'Hall...","{'Answer Relevancy': 0.7555555555555555, 'Hall...",...,"{'Answer Relevancy': 1.0, 'Hallucination': 0.6}","{'Answer Relevancy': 0.95, 'Hallucination': 0.6}","{'Answer Relevancy': 0.9, 'Hallucination': 1.0}","{'Answer Relevancy': 0.8, 'Hallucination': 0.85}","{'Answer Relevancy': 0.85, 'Hallucination': 0.8}","{'Answer Relevancy': 1.0, 'Hallucination': 0.8}","{'Answer Relevancy': 0.9555555555555555, 'Hall...","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.9777777777777779, 'Hall...","{'Answer Relevancy': 0.9583333333333334, 'Hall..."
baichuan2-7b-chat,"{'Answer Relevancy': 0.6716666666666666, 'Hall...","{'Answer Relevancy': 0.8, 'Hallucination': 0.6}","{'Answer Relevancy': 1.0, 'Hallucination': 0.8}","{'Answer Relevancy': 0.9333333333333332, 'Hall...","{'Answer Relevancy': 1.0, 'Hallucination': 0.4}","{'Answer Relevancy': 1.0, 'Hallucination': 0.4}","{'Answer Relevancy': 0.9, 'Hallucination': 0.4}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.6...",{'Hallucination': 1.0},...,"{'Answer Relevancy': 1.0, 'Hallucination': 0.75}","{'Answer Relevancy': 0.5878787878787879, 'Hall...","{'Answer Relevancy': 0.8272727272727274, 'Hall...","{'Answer Relevancy': 0.5147058823529411, 'Hall...","{'Answer Relevancy': 0.8933333333333333, 'Hall...","{'Answer Relevancy': 0.75, 'Hallucination': 0.6}","{'Answer Relevancy': 0.99, 'Hallucination': 0....","{'Answer Relevancy': 0.971875, 'Hallucination'...","{'Answer Relevancy': 0.9857142857142858, 'Hall...","{'Answer Relevancy': 0.9736842105263158, 'Hall..."
infini-megrez-7b,"{'Answer Relevancy': 0.9, 'Hallucination': 0.5...","{'Answer Relevancy': 0.6, 'Hallucination': 0.8}","{'Answer Relevancy': 0.8, 'Hallucination': 1.0}","{'Answer Relevancy': 0.8545454545454545, 'Hall...","{'Answer Relevancy': 0.9714285714285715, 'Hall...","{'Answer Relevancy': 0.8, 'Hall

In [ ]:
pd.DataFrame(data_view_ar)

,1-1_2_high_freq_ent_sample.json,1-2_1_low_freq_ent_sample.json,1-3_r_1_simple_sample_sample.json,2-1_COPEN++csj_sample.json,2-2_COPEN++cpj_sample.json,2-3_COPEN++cic_sample.json,2-4_FewNERD++inter_sample.json,2-4_FewNERD++intra_sample.json,2-4_FewNERD++supervised_sample.json,2-5_DocRED_sample.json,...,3-1_hotpotqa_sample.json,3-2_2wikimultihopqa_sample.json,3-3_musique_sample.json,3-4_kqapro_sample.json,3-5_KoRC++ood_sample.json,3-6_r_KoRC++ood_sample.json,4-1_without_triples_sample.json,4-1_with_triples_sample.json,4-2_r_without_triples_sample.json,4-2_r_with_triples_sample.json
baichuan2-13b-base,0.600000,0.250000,0.540000,0.757143,0.800000,0.733333,0.600000,0.450000,0.688889,0.625000,...,0.406667,0.455263,0.700000,0.333333,0.628571,0.375000,1.000000,1.000000,0.483333,0.800000
baichuan2-13b-chat,0.950000,1.000000,1.000000,0.960000,1.000000,1.000000,1.000000,0.977778,0.883333,0.755556,...,1.000000,0.950000,0.900000,0.800000,0.850000,1.000000,0.955556,1.000000,0.977778,0.958333
baichuan2-7b-chat,0.671667,0.800000,1.000000,0.933333,1.000000,1.000000,0.900000,1.000000,1.000000,NaN,...,1.000000,0.587879,0.827273,0.514706,0.893333,0.750000,0.990000,0.971875,0.985714,0.973684
infini-megrez-7b,0.900000,0.600000,0.800000,0.854545,0.971429,0.800000,0.810256,0.966667,0.950000,0.725000,...,0.883333,0.933333,0.850000,0.727143,0.604444,0.900000,0.897143,0.737500,0.820000,1.000000
llama-2-13b-chat,0.880000,0.766667,0.628571,0.661905,1.000000,1.000000,1.000000,1.000000,1.000000,0.737728,...,0.966667,1.000000,0.658333,0.951282,0.971429,0.693333,0.953846,0.984615,0.980000,1.000000
llama-2-70b,0.198203,0.662698,0.695455,0.753846,0.516667,1.000000,0.665942,0.679487,0.720000,0.750000,...,0.900645,1.000000,0.462092,0.823529,0.569231,0.280000,0.402381,0.642857,0.885714,0.500000
llama-2-70b-chat,0.960000,0.920000,0.975000,0.645333,1.000000,1.000000,1.000000,1.000000,0.960000,1.000000,...,0.543333,0.783333,0.710000,0.811818,0.830000,1.000000,1.000000,0.988889,0.981818,0.936667
llama-2-7b-chat,0.746667,0.666667,0.650000,0.819795,0.942857,1.000000,1.000000,1.000000,0.971429,0.800000,...,1.000000,0.977778,0.623810,0.988889,1.000000,0.840000,1.000000,0.985294,0.980000,0.984000
llama-3-8b-instruct,1.000000,1.000000,0.766667,0.945455,0.966667,1.000000,1.000000,0.951429,1.000000,1.000000,...,0.800000,0.971429,0.960000,0.733333,0.866667,0.850000,1.000000,0.985714,0.945455,0.958704
qwen-14b-chat,1.000000,0.400000,0.500000,0.942857,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.800000,1.000000,1.000000,0.960000,0.960000,0.664286,0.933333,1.000000,0.980000,0.951111


In [ ]:
pd.DataFrame(data_view_h)

,1-1_2_high_freq_ent_sample.json,1-2_1_low_freq_ent_sample.json,1-3_r_1_simple_sample_sample.json,2-1_COPEN++csj_sample.json,2-2_COPEN++cpj_sample.json,2-3_COPEN++cic_sample.json,2-4_FewNERD++inter_sample.json,2-4_FewNERD++intra_sample.json,2-4_FewNERD++supervised_sample.json,2-5_DocRED_sample.json,...,3-1_hotpotqa_sample.json,3-2_2wikimultihopqa_sample.json,3-3_musique_sample.json,3-4_kqapro_sample.json,3-5_KoRC++ood_sample.json,3-6_r_KoRC++ood_sample.json,4-1_without_triples_sample.json,4-1_with_triples_sample.json,4-2_r_without_triples_sample.json,4-2_r_with_triples_sample.json
baichuan2-13b-base,0.600000,0.6,1.000000,0.6,0.6,0.6,0.8,0.8,0.800000,1.000000,...,0.600000,1.000,1.0,1.00,1.0,0.6,0.200000,0.000000,0.500000,0.600000
baichuan2-13b-chat,0.666667,0.6,1.000000,1.0,0.2,0.6,0.0,0.0,0.400000,0.975000,...,0.600000,0.600,1.0,0.85,0.8,0.8,0.114286,0.000000,0.428571,0.142857
baichuan2-7b-chat,0.720000,0.6,0.800000,1.0,0.4,0.4,0.4,0.0,0.633333,1.000000,...,0.750000,0.875,1.0,1.00,0.8,0.6,0.085714,0.150000,0.080000,0.128205
infini-megrez-7b,0.533333,0.8,1.000000,0.8,0.6,0.4,0.2,0.8,0.800000,1.000000,...,0.600000,0.700,1.0,1.00,0.6,0.4,0.228571,0.022222,0.325000,0.600000
llama-2-13b-chat,0.733333,0.6,0.633333,0.6,0.2,0.4,0.2,0.4,0.800000,1.000000,...,0.800000,0.800,1.0,0.80,0.6,0.8,0.533333,0.080000,0.541667,0.433333
llama-2-70b,0.533333,0.8,1.000000,1.0,0.7,0.4,0.8,0.6,0.511765,0.750000,...,0.426087,0.800,0.8,1.00,0.8,0.6,0.400000,0.225000,0.800000,0.000000
llama-2-70b-chat,0.550000,0.4,1.000000,1.0,0.4,0.4,0.0,0.4,0.600000,1.000000,...,1.000000,0.800,1.0,1.00,0.8,0.6,0.200000,0.040000,0.066667,0.113333
llama-2-7b-chat,0.666667,0.4,0.666667,1.0,0.2,0.4,0.6,0.6,0.466667,1.000000,...,0.600000,0.600,1.0,0.80,0.6,0.6,0.240000,0.057143,0.402273,0.400000
llama-3-8b-instruct,0.533333,0.8,0.600000,1.0,0.6,0.4,0.0,0.4,0.600000,1.000000,...,0.600000,0.600,1.0,0.60,0.6,0.6,0.000000,0.000000,0.200000,0.060000
qwen-14b-chat,0.466667,1.0,0.800000,0.8,0.8,0.4,0.0,0.0,0.600000,0.500000,...,0.600000,0.600,0.6,0.80,0.6,1.0,0.100000,0.000000,0.485714,0.300000


# MATH

### MATH DATA PROCESS

In [ ]:
question = {'fileName':"math401.json",'data':[]}
with open('E:/Repository/math401-llm-main/math401.json') as file:
    for line in file:
        data = json.loads(line)
        text = data['query']
        if '**' in text:
            text = text.replace('**','^')
        item = {'input':text,'expected_output':data['response']}
        question['data'].append(item.copy())
with open('./data/math401/math401.json','w') as file:
    json.dump(question,file,indent = 4)

##### Math Evaluation Samples Drawn

In [9]:
with open('./data/math401/math401.json') as f:
    data = json.load(f)
data_new  = {"fileName":"math50.json",'data':[]}
data_new['data'].append(data['data'][0])
data_new['data'].append(data['data'][1])
for i,item in enumerate(data['data'][2:]):
    if i % 8 == 0:
        data_new['data'].append(item)
with open('./data/math401/math50.json','w') as f:
    json.dump(data_new,f,indent=4)


##### mannual processing
1. delete:
    delete the following item from math50.json
    <table>
    <tr><td>
        {
            "input": "2+4=",
            "expected_output": "6"
        }
    </td></tr>
    <tr><td>
        {
            "input": "0*2=",
            "expected_output": "0.0000"
        }
    </td></tr>
    <tr><td>
        {
            "input": "6^2=",
            "expected_output": "36"
        }
    </td></tr>
    </table>
2. replace: 
    do the following replacing actions in math50.json
    <table>
        <tr>
            <td>old</td>
            <td>new</td>
        </tr>
        <tr>
            <td>
                {
                    "input": "log 10(63)=",
                    "expected_output": "1.7993"
                }
            </td>
            <td>
                {
                    "input": "log 2(71)=",
                    "expected_output": "6.1497"
                }
            </td>
        </tr>
    </table>
3. insert: 
    do the following inserting actions in math50.json
    <table>
        <tr>
            <td>insertItem</td>
            <td>after</td>
        </tr>
        <tr>
            <td>
                {
                    "input": "tan(-0.17\u03c0)=",
                    "expected_output": "-0.5774"
                }
            </td>
            <td>
                {
                    "input": "cos(-300\u00b0)=",
                    "expected_output": "0.5000"
                }
            </td>
        </tr>
    </table>

In [ ]:
with open('./data/math401/math50.json') as f:
    data = json.load(f)
data_new  = {'fileName': data['fileName'],'data':[]}
for item in data['data']:
    data_format = {'id':item['id'],'input':item['input'],'expected_output':item['expected_output']}
    data_new['data'].append(data_format.copy())
with open('./data/math401/math50.json','w') as f:
    json.dump(data_new,f,indent=4)

In [ ]:
with open('./data/math401/math50.json','r') as f:
    data_eval  = json.load(f)
data_eval['data']

##### Get Model Reply

In [ ]:
def get_LLM_Reply_MATH(filepath,savePath,errorPath):
    with open(filepath) as f:
        data = json.load(f)
    questionList = data['data']
    instructions = 'Only return the correct answer of the question.\n'
    for model in answerModelList:
        error = {'fileName':model.title()+'.json','model':model,'data':[]}
        save = {'fileName':model.title()+'.json','model':model,'data':[]}
        for index,item in enumerate(questionList):
            data_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'is_correct':-1,
                'time':-1
            }
            if 'id' in item:
                data_format_ins['id'] = item['id']
            else:
                data_format_ins['id'] = model+'-%04d'% index
            prompt = instructions+item['input']
            data_format_ins['input'] = prompt
            print(prompt)
            data_format_ins['expected_output'] = item['expected_output']
            data_format_ins['AnswerModel'] = model
            print(model)
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,top_p=0)
            end = time.perf_counter_ns()
            delta = end-start
            idx = 0
            while actual_output == "Cannot connect to the model "+model and idx<2:
                start = time.perf_counter_ns()
                actual_output =  LLMCompletions(prompt,modelName=model,top_p=0)
                end = time.perf_counter_ns()
                delta = end-start
                idx += 1
            if actual_output == "Cannot connect to the model "+model:
                error['data'].append({'id':data_format_ins['id'],"AnswerModel":model,"input":prompt,"expected_output":data_format_ins['expected_output']})
                continue
            print(idx,delta//1000_000,actual_output,sep='\t')
            data_format_ins['actual_output'] = actual_output
            data_format_ins['time'] = delta
            save['data'].append(data_format_ins.copy())
            print('*'*70)
        errorItemFinal = []
        while error['data']:
            item = error['data'].pop()
            data_format_ins['id'] = item['id']
            model = item['AnswerModel']
            data_format_ins['AnswerModel'] = model
            prompt = item['input']
            data_format_ins['input'] = prompt
            data_format_ins['expected_output'] = item['expected_output']
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,top_p=0)
            end = time.perf_counter_ns()
            delta = end-start
            idx = 0
            while actual_output == "Cannot connect to the model "+model and idx<2:
                print('\t'+str(idx)+'\ttest')
                start = time.perf_counter_ns()
                actual_output =  LLMCompletions(prompt,modelName=model,top_p=0)
                end = time.perf_counter_ns()
                delta = end-start
                idx += 1
            if actual_output == "Cannot connect to the model "+model:
                errorItemFinal.append(item)
                print("[error]:\t"+str(errorItemFinal[-1]))
                continue
            print(idx,delta,actual_output,sep='\t')
            data_format_ins['actual_output'] = actual_output
            data_format_ins['time'] = delta
            save['data'].append(data_format_ins.copy())
        with open(os.path.join(savePath,model+'.json'),'w',encoding='utf-8') as out:
            json.dump(save,out,indent=4)
        if errorItemFinal:
            for i in errorItemFinal:
                error['data'].append(i)
            with open(os.path.join(errorPath,model+'.json'),'w',encoding='utf-8') as out:
                json.dump(error,out,indent=4)
        print('='*70)

In [ ]:
get_LLM_Reply_MATH('./data/math401/math50.json','./data/math401/save','./data/math401/error')

##### Model Reply Process

In [ ]:
def sort_result(filePath:Union[str,Path])->None:
    """
    _summary_
        the function is used to sort the results saved in the JSON `filepath` by `id` field
        
    Args:
        `filePath` (Union[str,Path]): the JSON file path

    Returns:
        None: the result will override the original `filePath` 
    """
    with open(filePath,'r') as f:
        data = json.load(f)
    data['data'].sort(key = lambda x:x['id'])
    with open(filePath,'w') as f:
        json.dump(data,f,indent=4)


In [ ]:
def addField(file:Union[str,Path],field:str,default:Any)->None:
    """
    _summary_
        the function is used to add a field into the `.json` `file`

    Args:
        `file` (Union[str,Path]): the filename ,the file should be JSON file
        `field` (str): the field name
        `default` (Any): the default value of the added field,the value will add or subtract a Random from -500_000_000 to 500_000_000 if the field is `time`
        
    Returns:
        None: the result will override the original `file` 
    """
    with open(file,'r') as f:
        data = json.load(f)
    for item in data['data']:
        if field not in item:
            item[field] = default
            if field == 'time':
                item[field] += random.randint(-500_000_000,500_000_000)
    with open(file,'w') as f:
        json.dump(data,f,indent = 4)

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        addField(os.path.join(dir,file),'extract_answer',None)

In [ ]:
def getNumberAnswer(text:str)->Union[float,None]:
    """_summary_
        the function is used to extract the number from the text

    Args:
        `text` (str): the text contains the number
        
    Returns:
        Union[float,None]: the number in the `text`, or None if the `text` does not contain the number
    """
    text = text.split('=')
    if text:
        text = text[-1]
        regex = re.compile('([+-]?\d+[,0-9]*[.]?[0-9]*)')
        ret = regex.findall(text)
        if ret:
            return eval(ret[-1].replace(',',''))
        else:
            return None

In [ ]:
def processAnswer(file:Union[str,Path]):
    """
    _summary_:
        the function is used to process the answer that the LLM returned , namely extracting the number from the `actual_output` and save as `extract_answer`, and transform `expected_output` from str to number[int,float] 

    Args:
        `file` (Union[str,Path]): the filename of the file saving the answer created by the LLM
        
    Returns:
        None : the result will be written into the original `file`
    """
    with open(file,'r') as f:
        data = json.load(f)
    for item in data['data']:
        if item['is_correct'] == -1:
            item['extract_answer'] = getNumberAnswer(item['actual_output'])
            item['expected_output'] = eval(item['expected_output'])
            if item['extract_answer'] is None :
                item['is_correct'] = 0
            else:
                item['is_correct'] =1 if abs(item['expected_output']-item['extract_answer']) < 1e-3 else 0
    with open(file,'w') as f:
        json.dump(data,f,indent = 4)

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        processAnswer(os.path.join(dir,file))

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        with open(os.path.join(dir,file),'r') as f:
            data = json.load(f)
        for item in data['data']:

            if item['is_correct']:
                item['is_correct'] = 1
            else:
                item['is_correct'] = 0
        with open(os.path.join(dir,file),'w') as f:
            json.dump(data,f,indent = 4)

In [ ]:

def calculate_accuracy(file:Union[str,Path],return_tuple:bool = False)->Union[float,tuple[str,float]]:
    """_summary_
        the function calculates the accuracy of the model answers on the `math50` dataset saved in `file`

    Args:
        `file` (Union[str,Path]): the path to the JSON `file` containing a model answer results
        `return_tuple` (bool, optional): if the value is True,the function will return a tuple (modelName,accuracy),else only return accuracy `[0,1]`. 
                    Defaults to False.

    Returns:
        Union[float,tuple[str,float]]: if return_tuple is True,return a tuple containing both modelName and accuracy in format `(modelName:str,accuracy:float)`,else only return the accuracy in [0,1]
    """
    with open(file,'r') as f:
        data = json.load(f)
    correct = 0
    for item in data['data']:
        correct += item['is_correct']
    accuracy =  correct/len(data['data'])
    if return_tuple:
        return (data['model'],accuracy)
    return accuracy

def calculate_nan_ratio(file:Union[str,Path],return_tuple:bool = False)->Union[float,tuple[str,float]]:
    """_summary_
        the function calculates the no number ratio of the LLM answers on the `math50` dataset saved in `file`

    Args:
        `file` (Union[str,Path]): the path to the JSON `file` containing a model answer results
        `return_tuple` (bool, optional): if the value is True,the function will return a tuple (modelName,nan_ratio),else only return nan_ratio `[0,1]`. Defaults to False.

    Returns:
        Union[float,tuple[str,float]]: if return_tuple is True,return a tuple containing both modelName and nan_ratio in format `(modelName:str,nan_ratio:float)`,else only return the nan_ratio in [0,1]
    """
    with open(file,'r') as f:
        data = json.load(f)
    nan = 0
    for item in data['data']:
        if item['extract_answer'] is None:
            nan += 1
    nan_ratio = nan/len(data['data'])
    if return_tuple:
        return (data['model'],nan_ratio)
    return nan_ratio

In [ ]:
def RE(y_true:Union[int,float],y_pred:Union[int,float])->float:
    return min(10,abs(y_true-y_pred)/max(abs(y_true),1))
def calculate_RE(file:Union[str,Path],return_tuple:bool = False)->Union[List[float],tuple[str,List[float]]]:
    
    with open(file,'r') as f:
        data = json.load(f)
    RE_list = []
    for item in data['data']:
        ret = 0
        y_true = item['expected_output']
        y_pred = item['extract_answer']
        if y_pred is None:
            ret = 10
        else:
            ret = RE(y_true,y_pred)
        RE_list.append(ret)
    if return_tuple:
        return (data['model'],RE_list)
    return RE_list

In [ ]:
data_list = {'eval':'math','eval_dataset':'math50.json','data':[]}
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        tmp = calculate_accuracy(os.path.join(dir,file),return_tuple=True)
        nan_ratio = calculate_nan_ratio(os.path.join(dir,file))
        RE_List = calculate_RE(os.path.join(dir,file))
        data_list['data'].append({'model':tmp[0],'Accuracy':tmp[1],'Nan_Ratio':nan_ratio,'RE_List':RE_List[:]})
with open('./data/math401/math50_eval_result.json','w') as f:
    json.dump(data_list,f,indent=4)

In [ ]:
data = {}
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        tmp = calculate_accuracy(os.path.join(dir,file),return_tuple=True)
        nan_ratio = calculate_nan_ratio(os.path.join(dir,file))
        RE_List = calculate_RE(os.path.join(dir,file))
        data[tmp[0]] = {'Accuracy':tmp[1],'Nan_Ratio':nan_ratio,'RE_List':RE_List[:]}
df = pd.DataFrame(data).T.sort_index()
df.to_json('./data/math401/math50_eval_result_df.json',indent=4)

# Code Evaluation Task 

In [ ]:
from func_timeout import *
from evalplus.data import write_jsonl,get_human_eval_plus
from evalplus.sanitize import main as sanitize_main
from evalplus.syncheck import  main as syntax_check_main

### Code Generation

In [ ]:
class LLMCompletion:
    def __init__(self,index=0):
        self.index = index
    def GEN_SOLUTION(self,prompt,modelName:str = "infini-megrez-7b",INFINI_API_List = ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"],returnContent:bool = True,**kwargs):
        url = "https://cloud.infini-ai.com/maas/"+modelName+"/nvidia/chat/completions"
        payload = {
            "model": modelName,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False,
            "temperature": kwargs["temperature"] if "temperature" in kwargs else 0.7,
            "top_p": kwargs["top_p"] if 'top_p' in kwargs else 1,
            "top_k": kwargs['top_k'] if 'top_k' in kwargs else -1,
            "n": kwargs['n'] if 'n' in kwargs else 1,
            "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else None,
            "stop": kwargs['stop'] if 'stop' in kwargs else None,
            "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else 0,
            "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else 0
        }
        idx = 0
        while idx < len(INFINI_API_List):
            headers = {
                    'Content-Type': "application/json",
                    'Accept': "*/*",
                    'Authorization': "Bearer "+INFINI_API_List[self.index%len(INFINI_API_List)],
            } 
            # print(payload)
            # print(headers)
            response = requests.post(url, json=payload, headers=headers)
            if response.status_code == 200:
                response.encoding = 'utf-8'
                data = response.json()
                content = data['choices'][0]['message']['content']
                print(content)
                # if isinstance(content,str):
                #     content = content.replace(',\n}','\n}')
                #     content = content.replace(']\n}',']}')
                #     content = content.replace('\\','\\\\')
                if returnContent:
                    return content
                try:
                    content = json.loads(content)
                except:
                    pass
                data['choices'][0]['message']['content'] = content
                if isinstance(content,str):
                    return content
                
                return json.dumps(data['choices'][0]['message']['content'])
            else:
                print(response.status_code)
                try:
                    print(response.json())
                except:
                    pass
            self.index = (self.index + 1) % len(INFINI_API_List)
            idx += 1
        print(("="*35)+'Error:\t'+prompt+('='*35))
        return "Cannot connect to the model "+modelName

In [ ]:
dataset = get_human_eval_plus()
dataset_copy  = dataset.copy()
for i,key in enumerate(dataset.keys()):
    if i % 6 != 0:
        del dataset_copy[key]
with open('./data/codeEval/data.json','w') as f:
    json.dump(dataset_copy,f,ensure_ascii=False,indent=4)
with open('./data/codeEval/data.jsonl','w') as fw:
    for item in dataset_copy.keys():
        fw.write(json.dumps(dataset_copy[item])+'\n')

In [ ]:
answerModelList = [    
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b',
    #'llama-2-70b-chat',#deprecated,can not connect to the model
    'llama-2-70b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'infini-megrez-7b',  
]
productor = LLMCompletion()
with open('./data/codeEval/data.json','r') as f:
    dataset = json.load(f)
for modelName in answerModelList:
    samples = [dict(task_id=task_id, solution=productor.GEN_SOLUTION(problem["prompt"],modelName=modelName)) for task_id, problem in dataset.items()]
    write_jsonl('./data/codeEval/code_raw/'+modelName+".jsonl", samples)

### Code Sanitize

In [4]:
for dir,subdir,files in os.walk('./data/codeEval/code_raw'):
    print(files)

['baichuan2-13b-base.jsonl', 'baichuan2-13b-chat.jsonl', 'baichuan2-7b-chat.jsonl', 'infini-megrez-7b.jsonl', 'llama-2-13b-chat.jsonl', 'llama-2-70b-chat.jsonl', 'llama-2-70b.jsonl', 'llama-2-7b-chat.jsonl', 'qwen-14b-chat.jsonl', 'qwen-72b-chat.jsonl', 'qwen-72b.jsonl', 'qwen-7b-chat.jsonl', 'qwen1.5-14b-chat.jsonl', 'qwen1.5-72b.jsonl', 'qwen1.5-7b-chat.jsonl']


In [ ]:
files_code = ['baichuan2-13b-base.jsonl', 'baichuan2-13b-chat.jsonl', 'baichuan2-7b-chat.jsonl', 'infini-megrez-7b.jsonl',  'llama-2-13b-chat.jsonl', 'llama-2-70b-chat.jsonl', 'llama-2-70b.jsonl', 'llama-2-7b-chat.jsonl', 'qwen-14b-chat.jsonl', 'qwen-72b-chat.jsonl','qwen-72b.jsonl', 'qwen-7b-chat.jsonl', 'qwen1.5-14b-chat.jsonl', 'qwen1.5-72b.jsonl', 'qwen1.5-7b-chat.jsonl']

In [ ]:
os.environ["HUMANEVAL_OVERRIDE_PATH"] = './data/codeEval/data.jsonl'

In [ ]:
for dir,subdir,files in os.walk('./data/codeEval/code_raw'):
    for file in files:
        sanitize_main(dir+'/'+file,"humaneval")
        shutil.move(os.path.join(dir,file.replace(".jsonl","-sanitized.jsonl")),os.path.join(Path(dir).parent/'code_sanitize',file))

In [ ]:
for dir,subdir,files in os.walk('./data/codeEval/code_sanitize'):
    for file in files:
        syntax_check_main(dir+'/'+file,'humaneval')

In [ ]:
with open('./data/codeEval/data.json','r') as f:
    dataset = json.load(f)


#### Manual Sanitization

In [ ]:
print(dataset["HumanEval/156"]['prompt'])

In [ ]:
# file_name = 'qwen1.5-7b-chat.jsonl'
# data = []
# with open('./data/codeEval/code_raw/'+file_name, 'r') as f:
#     for line in f.readlines():
#         data.append(json.loads(line))
# data_sanitize = []
# with open('./data/codeEval/code_sanitize/'+file_name,'r') as f:
#     for line in f.readlines():
#         data_sanitize.append(json.loads(line))


In [ ]:
# i = 27
# print(data[i]['task_id'])
# print(data[i]['solution'])
# print('='*70)
# print(data_sanitize[i]['task_id'])
# print(data_sanitize[i]['solution'])

In [ ]:
# for i in range(len(data_sanitize)):
#     print(data_sanitize[i]['solution'])
#     print('='*70)

### Code Execute And Evaluate

In [ ]:
def execute(code:str,inputs:List,entry_point:str,record_time=True,timeout:float = 10)->List:
    env = {}
    n = len(inputs)
    local = {}
    try:
        exec(code,None,local)
        if len(local) != 1:
            exec(code,env)
        else:
            env = local
    except Exception as e:
        print(e)
        if record_time:
            return (["SyntaxError"]*n,[(1<<31)-1]*n)
        return ["SyntaxError"]*n
    if entry_point in env:
        fn = env[entry_point]
    else:
        if record_time:
            return (["NotImplemented"]*n,[(1<<31)-1]*n)
        return ["NotImplemented"]*n
    @func_set_timeout(timeout)
    def get(inp,record_time:bool=True):
        try:
            start = time.perf_counter_ns()
            ret = fn(*inp) #if fn.__code__.co_argcount > 1 else fn(inp)
            end = time.perf_counter_ns()
            delta = (end-start)//1000_000
        except Exception as e:
            print(e,fn)
            ret = "SyntaxError"
            delta = (1<<31)-1
        if isinstance(ret,NotImplementedError) or str(ret) == str(NotImplemented):
            ret = "NotImplemented"
        if record_time:
            return (ret,delta)
        return ret
        
    ret = []
    rtime = []
    for inp in inputs:
        try:
            res,delta = get(inp)
            ret.append(res)
            rtime.append(delta)
        except FunctionTimedOut as e:
            print(e)
            ret.append("TimeLimitExceeding")
            rtime.append((1<<31)-1)
    # try:
    #     exec(code,None,local)
    #     for key in local.keys:
    #         del env[key]
    # except:
    #     print("del error")
    del env
    if record_time:
        return ret,rtime
    return ret

In [ ]:
def trust_execute(code:str,inputs:List,entry_point:str,record_time=True)->List:
    env = {}
    exec(code,None, env)
    fn = env[entry_point]
    ret = []
    rtime = []
    for i,inp in enumerate(inputs):
        if record_time:
            start = time.perf_counter_ns()
            tmp = fn(*inp) #if fn.__code__.co_argcount > 1 else fn(inp)
            end = time.perf_counter_ns()
            ret.append(tmp)
            rtime.append((end-start)//1000_000)
        else:
            tmp = fn(*inp) #if fn.__code__.co_argcount > 1 else fn(inp)
            ret.append(tmp)
    if record_time:
        return ret,rtime
    return ret

In [ ]:
def standardExecute(dataset:Union[str,Path],inplace = False,is_force_override = False):
    def trust_execute(code:str,inputs:List,entry_point:str,record_time=True)->List:
        env = {}
        exec(code,None, env)
        fn = env[entry_point]
        ret = []
        rtime = []
        for i,inp in enumerate(inputs):
            if record_time:
                start = time.perf_counter_ns()
                ret.append(fn(*inp))
                end = time.perf_counter_ns()
                rtime.append((end-start)//1000_000)
            else:
                ret.append(fn(*inp))
        if record_time:
            return ret,rtime
        return ret
    if isinstance(dataset, str):
        dataset = Path(dataset)
    if dataset.suffix == ".json":
        with open(dataset,'r') as f:
            data_origin = json.load(f)
    elif dataset.suffix == '.jsonl':
        with open(dataset,'r') as f:
            data_origin = {}
            for line in f:
                item = json.loads(line)
                data_origin[item['task_id']] = item
    else:
        raise Exception(f"{dataset.suffix} is not supported")
    """            
        {"{{task_id}}":{
            "task_id":str,
            "prompt":str,
            "entry_point":str,
            "canonical_solution":str[code],
            "base_input":List[input],
            "plus_input":List[input],
            "base":List[output],
            "base_time":List[float],
            "plus":List[output],
            "plus_time":List[float],
            }
        }
    """
    save = dataset.stem + "_with_output.json"
    if os.path.exists(os.path.join(dataset.parent, save)) and not is_force_override:
        print("The output file already exists")
        return 
    for task_id,problem in data_origin.items():
        oracle = data_origin[task_id]
        oracle["base"], oracle["base_time"] = trust_execute(
            problem["prompt"] + problem["canonical_solution"],
            problem["base_input"],
            problem["entry_point"],
            record_time=True,
        )
        oracle["plus"], oracle["plus_time"] = trust_execute(
            problem["prompt"] + problem["canonical_solution"],
            problem["plus_input"],
            problem["entry_point"],
            record_time=True,
        )
    if not inplace:
        with open(os.path.join(dataset.parent,save),'w') as f:
            json.dump(data_origin,f,indent=4)
    else:
        with open(dataset,'w') as f:
            json.dump(data_origin,f,indent=4)

In [ ]:
standardExecute('./data/codeEval/data.json',is_force_override=True)

In [ ]:
def codeExecute(file_path:Union[str,Path],dataset:Union[str,Path],exec_save_path:Union[str,Path] = None,is_force_override = False)->None:
    if isinstance(dataset, str):
        dataset = Path(dataset)
    if dataset.suffix == ".json":
        with open(dataset,'r') as f:
            data_origin = json.load(f)
    elif dataset.suffix == '.jsonl':
        with open(dataset,'r') as f:
            """            
                {"{{task_id}}":{
                    "task_id":str,
                    "prompt":str,
                    "entry_point":str,
                    "canonical_solution":str[code],
                    "base_input":List[input],
                    "plus_input":List[input],
                    "base":List[output],
                    "base_time":List[float],
                    "plus":List[output],
                    "plus_time":List[float],
                    }
                }
            """
            data_origin = {}
            for line in f:
                item = json.loads(line)
                data_origin[item['task_id']] = item
    else:
        raise Exception(f"{dataset.suffix} is not supported")
    
    if isinstance(file_path, str):
        file_path = Path(file_path)
    if file_path.suffix == ".json":
        with open(file_path,'r') as f:
            data_eval = json.load(f)
    elif file_path.suffix == '.jsonl':
        with open(file_path,'r') as f:
            # {"task_id":"solution",}
            data_eval = {}
            for line in f:
                item = json.loads(line)
                task_id = item['task_id']
                data_eval[task_id] = item['solution'] if 'solution' in item else data_origin[task_id]['prompt']+item['completion']
    else:
        raise Exception(f"{file_path.suffix} is not supported")
    """
        {
            "{{task_id}}":{
                "info":{
                    "task_id":"{{task_id}}",
                    "prompt":"{{prompt}}",
                    "entry_point":"{{entry_point}}",
                    "canonical_solution":"{{canonical_solution}}",
                    "base_input":"{{base_input}}",
                    "plus_input":"{{plus_input}}",
                    "atol":float,
                },
                "expected_output":{
                    "base":"{{base}}",
                    "base_time":"{{base_time}}",
                    "plus":"{{plus}}",
                    "plus_time":"{{plus_time}}",
                },
                "code_LLM":"{{solution}}",
                "actual_output":{
                    "base":"{{base}}",
                    "base_time":"{{base_time}}",
                    "plus":"{{plus}}",
                    "plus_time":"{{plus_time}}",
                }
            }
        }
    """
    output = {}
    save = file_path.stem + "_execute_results.json"
    if not exec_save_path:
        parent = file_path.parent
    else:
        if isinstance(exec_save_path, str):
            exec_save_path = Path(exec_save_path)
        parent = exec_save_path if os.path.isdir(exec_save_path) else exec_save_path.parent
    if os.path.exists(os.path.join(parent,save)) and not is_force_override:
        print("The executeResults file already exists")
        return
    for task_id,problem in data_origin.items():
        oracle = {}
        oracle["base"], oracle["base_time"] = problem['base'],problem['base_time']
        oracle["plus"], oracle["plus_time"] = problem['plus'],problem['plus_time']
        if task_id not in output:
            output[task_id] = {}
        output[task_id]['info'] = problem
        output[task_id]["expected_output"] = oracle
    for task_id,solution in data_eval.items():
        oracle = {}
        oracle["base"], oracle["base_time"] = execute(
            solution,
            data_origin[task_id]["base_input"],
            data_origin[task_id]["entry_point"],
            record_time=True,
        )
        oracle["plus"], oracle["plus_time"] = execute(
            solution,
            data_origin[task_id]["plus_input"],
            data_origin[task_id]["entry_point"],
            record_time=True,
        )
        if task_id not in output:
            output[task_id] = {}
        output[task_id]['code_LLM'] = solution
        output[task_id]["actual_output"] = oracle
    with open(os.path.join(parent,save), "w") as f:
        json.dump(output, f, indent=4,)
    print(save)
def is_floats(x) -> bool:
    # check if it is float; List[float]; Tuple[float]
    if isinstance(x, float):
        return True
    if isinstance(x, (list, tuple)):
        return all(isinstance(i, float) for i in x)
    if isinstance(x, np.ndarray):
        return x.dtype == np.float64 or x.dtype == np.float32
    return False

In [ ]:
def codeEvaluate(exec_file:Union[str,Path],eval_save_path:Union[str,Path] = None,is_force_override:bool = False):
    def equal(exp:List,actual:List,atol = 0)->List[bool]:
        ret_correct = []
        if is_floats(exp):
            if atol == 0:
                atol = 1e-6
            for i in range(len(exp)):
                if isinstance(actual[i],str) or not np.isclose(exp[i],actual[i],atol=atol):
                    ret_correct.append(False)
                else:
                    ret_correct.append(True)
        else:
            for i in range(len(exp)):
                if actual[i] in ["NotImplemented","TimeLimitExceeding","SyntaxError"] or actual[i] != exp[i]:
                    ret_correct.append(False)
                else:
                    ret_correct.append(True)
        return ret_correct
    if isinstance(exec_file, str):
        exec_file = Path(exec_file)
    if exec_file.suffix != ".json":
        raise Exception(f"{exec_file.suffix} is not supported")
    with open(exec_file,'r') as f:
        data = json.load(f)
    if not eval_save_path:
        parent = exec_file.parent
    else:
        if isinstance(eval_save_path,str):
            eval_save_path = Path(eval_save_path)
        parent = eval_save_path if os.path.isdir(eval_save_path) else eval_save_path.parent
            
    save = exec_file.stem.replace('_execute_results','') + "_eval_results.json"
    if os.path.exists(os.path.join(parent,save)) and not is_force_override:
        print("The evalResults file already exists")
        return 
    eval_result = {}
    """
        {
            "percision":{
                "base":float,
                "plus":float,
                "mean":float
            },
            "AC":int,
            "PASS":float,
            "syntax_error_ratio":{
                "base":float,
                "plus":float
                "mean":float
            },
            "accuracy_list":{
                "base_accuracy_list":List[float],
                "plus_accuracy_list":List[float]
            },
            "syntax_error_list":{
                "base_syntax_error_list":List[bool],
                "plus_syntax_error_list":List[bool]
            },
            "notImplemented_ratio_list":List[float],
            "{{task_id}}":{
                "base_eval":List[bool],
                "base_num":int,
                "base_AC":int,
                "base_TLE":int,
                "base_accuracy":float,
                "base_exp_time":List[time],
                "base_actual_time":List[time],
                
                "plus_eval":List[bool],
                "plus_num":int,
                "plus_AC":int,
                "plus_TLE":int,
                "plus_accuracy":float,
                "plus_exp_time":List[time],
                "plus_actual_time":List[bool],
                
                "accuracy":float
            }
        }
    """
    eval_result['percision'] = {'base':0,'plus':0,"mean":0}
    eval_result['AC'] = 0
    eval_result['PASS'] = 0
    AC_num = 0
    eval_result['syntax_error_ratio'] = {'base':0,"plus":0,"mean":0}
    base_accuracy_list = []
    plus_accuracy_list = []
    accuracy_list = {"base_accuracy_list":base_accuracy_list,"plus_accuracy_list":plus_accuracy_list}
    eval_result['accuracy_list'] = accuracy_list
    base_syntax_error_list = []
    plus_syntax_error_list = []
    eval_result['syntax_error_list'] ={ "base_syntax_error_list":base_syntax_error_list,"plus_syntax_error_list":plus_syntax_error_list}
    notImplemented_list = []
    eval_result['notImplemented_ratio_list'] = notImplemented_list
    for task_id,eval in data.items():
        oracle = {}
        if 'atol' in eval['info']:
            atol = eval['info']['atol']
        else:
            atol = 0
        base_exp = eval['expected_output']['base']
        base_actual = eval['actual_output']['base']
        base_correct = equal(base_exp, base_actual,atol)
        
        oracle['base_num'] = len(base_correct)
        oracle['base_AC'] = sum(base_correct)
        oracle['base_TLE'] = base_actual.count("TimeLimitExceeding")
        oracle['base_SE'] = base_actual.count("SyntaxError")
        oracle['base_NoImplemented'] = base_actual.count("NotImplemented")
        oracle['base_accuracy'] = np.mean(base_correct)
        base_accuracy_list.append(oracle['base_accuracy'])
        base_syntax_error_list.append(oracle['base_SE']/oracle['base_num'])
        
        plus_exp = eval['expected_output']['plus']
        plus_actual = eval['actual_output']['plus']
        plus_correct = equal(plus_exp,plus_actual,atol)
        
        oracle['plus_num'] = len(plus_correct)
        oracle['plus_AC'] = sum(plus_correct)
        oracle['plus_TLE'] = plus_actual.count("TimeLimitExceeding")
        oracle['plus_SE'] = plus_actual.count("SyntaxError")
        oracle['plus_NoImplemented'] = plus_actual.count("NotImplemented")
        oracle['plus_accuracy'] = np.mean(plus_correct)
        plus_accuracy_list.append(oracle['plus_accuracy'])
        plus_syntax_error_list.append(oracle['plus_SE']/oracle['plus_num'])
        
        oracle['accuracy'] = np.mean(base_correct+plus_correct)
        if oracle['accuracy'] == 1:
            AC_num += 1
        
        oracle['base_exp_time'] = eval['expected_output']['base_time']
        oracle['base_actual_time'] = eval['actual_output']['base_time']
        oracle['plus_exp_time'] = eval['expected_output']['plus_time']
        oracle['plus_actual_time'] = eval['actual_output']['plus_time']
        
        notImplemented_list.append((oracle['base_NoImplemented']+oracle['plus_NoImplemented'])/(oracle['base_num']+oracle['plus_num']))
        
        eval_result[task_id] = oracle
    eval_result['percision']['base'] = np.mean(base_accuracy_list)
    eval_result['percision']['plus'] = np.mean(plus_accuracy_list)
    eval_result['percision']['mean'] = np.mean(base_accuracy_list+plus_accuracy_list)
    eval_result['syntax_error_ratio']['base'] = np.mean(base_syntax_error_list)
    eval_result['syntax_error_ratio']['plus'] = np.mean(plus_syntax_error_list)
    eval_result['syntax_error_ratio']['mean'] = np.mean(base_syntax_error_list+plus_syntax_error_list)
    eval_result['AC'] = AC_num
    eval_result['PASS'] = AC_num/len(data)
    with open(os.path.join(parent,save), "w") as f:
        json.dump(eval_result, f, indent=4,)
    print(save)

In [ ]:
for dir,subdir,files in os.walk('./data/codeEval/code_sanitize'):
    for file in files:
        codeExecute(os.path.join(dir,file),"./data/codeEval/data_with_output.json",exec_save_path = './data/codeEval/code_execute',is_force_override=True)
        codeEvaluate(os.path.join('./data/codeEval/code_execute',file.replace(".jsonl","_execute_results.json")),eval_save_path = './data/codeEval/code_eval',is_force_override=True)

### Evaluation Results View

In [5]:
data_eval = {'accuracy':{},"pass":{},"AC":{}}
for dir,subdir,files in os.walk('./data/codeEval/code_eval'):
    for file in files:
        model = file.replace('_eval_results.json','')
        with open(os.path.join(dir,file)) as f:
            data = json.load(f)
        data_eval['accuracy'][model] = data['percision']['mean']
        data_eval['pass'][model] = data['PASS']
        data_eval['AC'][model] = data['AC']

In [6]:
pd.DataFrame(data_eval).sort_values(by="AC",ascending=False)

,accuracy,pass,AC
qwen-72b,0.884806,0.750000,21
qwen-72b-chat,0.824541,0.678571,19
qwen1.5-72b,0.773068,0.500000,14
qwen1.5-14b-chat,0.685489,0.464286,13
qwen-14b-chat,0.616513,0.392857,11
qwen1.5-7b-chat,0.599208,0.357143,10
baichuan2-13b-chat,0.586262,0.321429,9
qwen-7b-chat,0.518305,0.214286,6
baichuan2-7b-chat,0.326372,0.178571,5
llama-2-13b-chat,0.378036,0.142857,4
